In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from time import sleep
import re

In [ ]:
def get_harlequin_category_links(page_id=1, per_page=48):
    cats = []
    next_page = True

    while next_page == True:
        url = f"https://www.harlequin.com/genres-list.html?p={page_id}&ipp={per_page}&searchTerm=&last="
        page = requests.get(url).content
        soup = BeautifulSoup(page)
        if not soup.find("a", {"aria-label":"Next page"}):
            next_page = False
        blocks = soup.find_all("div", class_="column column-block")    
        links = [block.find("a") for block in blocks]
        links = [(link.get_text(),
                  re.search(r"[0-9é\(\)a-z-\s]+(?=\.html)", link.get("href")).group(0)) for link in links]
        cats.extend(links)
        page_id += 1
        sleep(1)
    
    return list(set(cats))

In [ ]:
def get_harlequin_category_ids(categories):
    ids = {}
    for category in categories:
        url = f"https://www.harlequin.com/shop/category/{category[1]}.html"
        page = requests.get(url).content
        soup = BeautifulSoup(page)
        id_num = re.search("(?<=categoryId\=)\d+", str(soup))
        try:
            id_num = int(id_num.group(0))
            ids[category[0]] = id_num
        except:
            print(category[0])
        sleep(1)
        
    return ids

In [ ]:
# Get the DB ID for each category

categories = get_harlequin_category_links()

In [ ]:
# Get the six-digit ID for each category

category_ids = get_harlequin_category_ids(categories)

In [ ]:
# Error on site search

category_ids["Ménage"] = "36203"

In [ ]:
# Save the category ids

with open("categories.txt", "w") as file:
    for x in category_ids.keys():
        file.write(f"{x} - {category_ids[x]}\n")